In [5]:
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt

In [6]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/home/etpaler/test/RHEO_benchmarks/surf_normal/sin_surface'

In [7]:
#parameter grids
var_sf_grid = {"0.05":"20","0.1":"10","0.2":"5","0.25":"4","0.5":"2"}
#var_sf_grid = {"0.1":"10"}
var_kernel_grid = {"Quintic":"5"}


# Generate input data file

Generate atom positions for sinusoidal surface 

In [8]:
# System size

for sf in var_sf_grid:
    sf = float(sf)
    
    ###Generate Atom Positions###
    
    L = 10.0 #System size
    res = int(L/sf) #resolution

    #System boundaries
    y_max = np.max(L+np.sin(2*np.pi/L*np.linspace(0,L,res)))
    y_min = np.min(np.sin(2*np.pi/L*np.linspace(0,L,res)))
    x_min = 0.0
    x_max = L

    type_pos_all = [] #Array to hold the atom type and position (x,y,z)

    #Top and bottom surfaces are sin curves
    for x in np.linspace(0,L,res):
        y_bot = np.sin(2*np.pi*x/L)
        #type_pos_all.append([2, [x, y_bot,0.0]]) #Bottom surface atoms
        #type_pos_all.append([2, [x, L + y_bot,0.0]]) #Top surface atoms
        for y in np.linspace(y_bot,L+y_bot, res):
            type_pos_all.append([1, [x,y,0.0]]) #Interior atoms

    n_atoms = len(type_pos_all)
    
    ###Write Data File###
    
    with open("sin_surface_" + str(sf) + ".data","w") as fdata:
        fdata.write("Sinusoidal surface atom positions\n\n")

        #---Header---#
        fdata.write("{} atoms\n".format(n_atoms)) #total atoms
        fdata.write("{} atom types\n\n".format(2)) #types of atoms

        #Box dimensions
        fdata.write("{} {} xlo xhi\n".format(x_min, x_max))
        fdata.write("{} {} ylo yhi\n".format(y_min, y_max))
        fdata.write("{} {} zlo zhi\n".format(-0.1*sf, 0.1*sf))
        fdata.write("\n")

        #---Atom Type Mass---#
        fdata.write("Masses \n\n")
        fdata.write("{} {} \n".format(1, 0.25))
        fdata.write("{} {} \n".format(2, 0.25))
        fdata.write("\n")

        #---Atoms---#
        fdata.write("Atoms # rheo \n\n")


        for count, atom in enumerate(type_pos_all):
            #fields_data_atom = (char *) "id type rho temp phase x"
            fdata.write("{} {} {} {} {} {} {} {} {} {} {}\n".format(count+1, atom[0], 1.0, 1.0, 0, *atom[1], 0, 0, 0))
        
        fdata.write("\n")

        #---Velocities---#
        fdata.write("Velocities\n\n")

        for count, atom in enumerate(type_pos_all):
            fdata.write("{} {} {} {}\n".format(count+1, 0.0, 0.0, 0.0))
            
        fdata.write("\n")


# Generate LAMMPS Input Scripts

For sin surface:

In [9]:
#Generate lammps files
filenames = []
for sf in var_sf_grid:
    for kernel in var_kernel_grid:
        run_code = "1_"+var_kernel_grid[kernel]+"_"+var_sf_grid[sf]
        filename = "in."+run_code+".lmp"
        file = shutil.copyfile("lammps_in_sin.template",pwd+"/"+filename)
        file_in = open(file,"rt")
        file_text = file_in.read()

        #Replace variables
        file_text = file_text.replace("var_sf",sf)
        file_text = file_text.replace("var_kernel",kernel)
        file_text = file_text.replace("var_data","sin_surface_"+sf+".data")
        file_text = file_text.replace("var_filename",run_code)

        file_out = open(file,"wt")
        file_out.write(file_text)
        file_out.close()
        
        filenames.append(filename)

In [10]:
filenames

['in.1_5_20.lmp',
 'in.1_5_10.lmp',
 'in.1_5_5.lmp',
 'in.1_5_4.lmp',
 'in.1_5_2.lmp']

# Write the Jobscript

### Solo

In [10]:
account="FY150028"
job_name="TestChannel"
time="24:0:0"
partition="batch"
nodes="1"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/lammps_rheo/lammps/build/lmp -in "

In [11]:
with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")

### WSL

In [16]:
run_command = "mpirun -np 2 ~/dev/rlammps_extrusion/build/lmp -in "

with open("jobscript","w") as jobscript:
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")